In [1]:
# import packages
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Embedding, Flatten, Dense, SimpleRNN, LSTM, GRU
from keras.models import Sequential
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [3]:
# 1. Import the data and tokenize to use with Keras
testfile = open('data/congress_test.csv', "r+",encoding='utf-8', errors='ignore')
text = testfile.read()
testfile.seek(0)
testfile.write(text)
testfile.close()

trainfile = open('data/congress_train.csv', "r+",encoding='utf-8', errors='ignore')
text = trainfile.read()
trainfile.seek(0)
trainfile.write(text)
trainfile.close()

valfile = open('data/congress_val.csv', "r+",encoding='utf-8', errors='ignore')
text = valfile.read()
valfile.seek(0)
valfile.write(text)
valfile.close()

# import the data
train_data = pd.read_csv('data/congress_train.csv').dropna()
val_data = pd.read_csv('data/congress_val.csv').dropna()
test_data = pd.read_csv('data/congress_test.csv').dropna()
train_text = list(train_data['Title'])
train_text = [str(i) for i in train_text]
val_text = list(val_data['Title'])
val_text = [str(i) for i in val_text]
test_text = list(test_data['Title'])
test_text = [str(i) for i in test_text]
label_train = to_categorical(list(train_data['Major']))
label_val = to_categorical(list(val_data['Major']))
label_test = to_categorical(list(test_data['Major']))

# tokenize the data
# Keep only the 10000 most frequent words
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(train_text)
train_seq = tokenizer.texts_to_sequences(train_text)
test_seq = tokenizer.texts_to_sequences(test_text)
val_seq = tokenizer.texts_to_sequences(val_text)
# Limit each bill's title to a maximum length of 100 words
# Pad each sequence to be of length 100
x_train = pad_sequences(train_seq, maxlen=100)
x_test = pad_sequences(test_seq, maxlen=100)
x_val = pad_sequences(val_seq, maxlen=100)

In [ ]:
# 3. Estimate a basic feed-forward network
basic_ff = Sequential()
basic_ff.add(Embedding(10000, 32, input_length=100))
basic_ff.add(Flatten())
basic_ff.add(Dense(24, activation='softmax'))
basic_ff.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
result_basic_ff = basic_ff.fit(x_train, label_train, validation_data=(x_val, label_val), \
                               epochs=50, batch_size=512, verbose=0)

In [ ]:
# 4. Estimate a recurrent neural network (RNN) with a layer_simple_rnn
rnn = Sequential()
rnn.add(Embedding(10000, 32, input_length=100))
rnn.add(SimpleRNN(32))
rnn.add(Dense(24, activation='softmax'))
rnn.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
result_rnn = rnn.fit(x_train, label_train, validation_data=(x_val, label_val), epochs=50, batch_size=512, verbose=0)

In [ ]:
# 5. Estimate an RNN with an LSTM layer
lstm = Sequential()
lstm.add(Embedding(10000, 32, input_length=100))
lstm.add(LSTM(32))
lstm.add(Dense(24, activation='softmax'))
lstm.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
result_lstm = lstm.fit(x_train, label_train, validation_data=(x_val, label_val), epochs=50, batch_size=512, verbose=0)